# Hello Flask

To get started, make sure we  need to install flask

```
pip install flask
```

Make sure you can run the `python app.py` 

## Flask Basics

### Bare Minimum

This is the starting point when making any flask app, which returns a static string whenever it is hit on `http://localhost:5000/`

```python
from flask import Flask


app = Flask(__name__)


@app.route('/')
def index():
	return "It's alive!"


if __name__ == '__main__':
	app.run(
		host = '127.0.0.1',
		port = 5000
	)
```

### Adding another route 

Each route that you add, such as `/api/do-this` or `/api/process-that`, would typically correspond to a python function. So to add another function `ruok` for the path `/ruok`:

```python
@app.route('/ruok')
def ruok():
	return 'yes...'
```

### Getting input for your function

A lot of functions that we write probably expects some sort of input, and input can usually be passed to the handler of the route using, _query string_ , _form data_ or _json data_.

So the example below, we are able to get input from the json data from the body of the `POST` request

```python
@app.route('/sqrt', methods=['POST'])
def sqrt():
    n = request.json['n']
    return jsonify(sqrt=model_cache[n])
```

Similarly, _form data_ and _query string_ parameters can be accessed using the `form` and `args` attributes respectively

```python
@app.route('/echo', methods=['POST'])
def echo():
    is_upper = request.args.get('uppercase', False)
    data = request.form['data']
    if is_upper:
        data = data.upper()
    else:
        data = data.lower()
    return jsonify({'data': data})
```

### Returning data

In typical APIs, our responses are in JSON, and we just `jsonify` 

```python
jsonify({'data': data})
jsonify(sqrt=5)
```

### Uploading an image 

To handle image uploads, we get the following code snippet. Note, _always_ use `secure_filename`

```python
@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['image']
    filename = secure_filename(file.filename)
    file.save(os.path.join('uploads', filename))
    return jsonify(status='success')
```

This will be able to handle this corresponding request

In [162]:
res = requests.post('http://localhost:5000/upload', files={
    'image': ('uploaded-image.jpg', open('sample-image.jpg', 'rb'))
})
res.json()

{'status': 'success'}

### Special note on handling user input

I guess it is good time to give a motivating example on why we should _always_ sanitize user input. If we did not use `secure_filename`

In [163]:
requests.post('http://localhost:5000/upload', files={
    'image': ('../uploaded-image.jpg', open('sample-image.jpg', 'rb'))
})

<Response [200]>

What about this?

In [ ]:
# requests.post('http://localhost:5000/upload', files={
#     'image': ('../app.py', open('exploit.py', 'rb'))
# })

### Path parameters and error codes

Here is a quick example of getting a parameter from the path as well as returning non-200 status codes.

```python
@app.route('/user/<user_id>')
def get_user(user_id):
    return jsonify(err="Can't find {}".format(user_id)), 404
```

In [165]:
res = requests.get('http://localhost:5000/user/my-user-id-0123')

In [166]:
assert res.status_code == 404

In [167]:
res.json()

{'err': "Can't find my-user-id-0123"}

## Practice with requests

For this part, make the appropriate on each cell the has the comment `#Edit me` so that all assertions are passed. This shouldn't you more than few minutes.

In [1]:
import requests

### Is it alive?

Make sure our server is [alive](http://localhost:5000)!

In [148]:
res = None #Edit me

In [149]:
res.raise_for_status()

AttributeError: 'NoneType' object has no attribute 'raise_for_status'

In [150]:
assert res.text == "It's alive!"

AttributeError: 'NoneType' object has no attribute 'text'

### ruok?

Make sure that our web app isn't down.

In [145]:
res = requests.post('http://localhost:5000/ruok'.format(url)) #Edit me

In [146]:
res.raise_for_status()

HTTPError: 405 Client Error: METHOD NOT ALLOWED for url: http://localhost:5000/ruok

In [147]:
assert res.text == 'yes...'

AssertionError: 

### Mathematics

Let's cheer up our flask server with some math questions. Also what happens we you ask it to process a bigger number? Maybe `n=16`? 

In [142]:
res = requests.post('http://localhost:5000/sqrt', data={'n': 9}) #Edit me

In [143]:
assert res.status_code == 200

AssertionError: 

In [144]:
assert res.json()['sqrt'] == 3

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Echo

Hype up the server!

In [138]:
res = requests.post('http://localhost:5000/echo', data={'data': "let's go"}) #Edit me

In [139]:
res.raise_for_status()

In [140]:
res.text

'{"data":"let\'s go"}\n'

In [141]:
assert res.json()['data'] == "LET'S GO"

AssertionError: 